In [3]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))
seq_model.summary()

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)
model = Model(input_tensor, output_tensor)
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12

7.1.2 多输入模型

In [4]:
# 用函数式 API 实现双输入问答模型
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question],axis=-1)

answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated) 

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [6]:
# 将数据输入到多输入模型中
import numpy as np
from keras import utils

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,size=(num_samples, max_length))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = utils.to_categorical(answers, answer_vocabulary_size)

model.fit([text, question], answers, epochs=10, batch_size=128)
model.fit({'text': text, 'question': question}, answers,
          epochs=10, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.2146 - acc: 1.0000e-03
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1993 - acc: 0.0470
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1706 - acc: 0.0490
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0672 - acc: 0.0070
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0123 - acc: 0.0060
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.9708 - acc: 0.0060
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.8838 - acc: 0.0080
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.7881 - acc: 0.0080
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.6971 - acc: 0.0110
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss

7.1.3 多输出模型

In [7]:
# 用函数式 API 实现一个三输出模型
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input,[age_prediction, income_prediction, gender_prediction])

In [8]:
# 多输出模型的编译选项：多重损失
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})

In [9]:
# 多输出模型的编译选项：损失加权
model.compile(optimizer='rmsprop',
 loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
 loss_weights=[0.25, 1., 10.])
model.compile(optimizer='rmsprop',
 loss={'age': 'mse',
 'income': 'categorical_crossentropy',
 'gender': 'binary_crossentropy'},
 loss_weights={'age': 0.25,
 'income': 1.,
 'gender': 10.})

In [ ]:
# 将数据输入到多输出模型中
model.fit(posts, [age_targets, income_targets, gender_targets],
 epochs=10, batch_size=64)
model.fit(posts, {'age': age_targets,
 'income': income_targets,
 'gender': gender_targets},
 epochs=10, batch_size=64) 